In [1]:
!git clone https://oramleo@github.com/DionisiusMayr/FreewayGame.git

Cloning into 'FreewayGame'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 178 (delta 84), reused 119 (delta 41), pack-reused 0
Receiving objects: 100% (178/178), 46.01 MiB | 7.49 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [2]:
!cp -r FreewayGame/dionisius.mayr/* FreewayGame/leonardo/


In [4]:
!pip install gym
!pip install gym[atari]


In [8]:
import sys
sys.path.append('/content/FreewayGame/leonardo/')
%matplotlib inline
from collections import defaultdict
from typing import List

import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots

In [9]:
def print_result(i, scores, total_reward, score):
    if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

In [10]:
RAM_mask = [
      14  # Chicken Y
    , 16  # Chicken Lane Collide
    , 108, 109, 110, 111, 112, 113, 114, 115, 116, 117  # Car X Coords
]

In [11]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3

    return ob
    
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

In [57]:
class SarsaLFA():
    def __init__(self, gamma: float, available_actions: int, discount_factor: int, alpha: float):
        self.gamma = gamma
        self.available_actions = available_actions
        self.N0 = N0
        self.alpha = alpha

        self.weights = np.random.rand(2+len(RAM_mask))

        self.state_visits = defaultdict(lambda: 0)

    def qw(self, state, action):
        return np.dot(self.get_features(state, action), self.weights)

    def get_features(self, state, action):
        return np.append(np.frombuffer(state, dtype=np.uint8, count=-1), [action, 1])

    def act(self, state):
        epsilon = self.N0 / (self.N0 + self.state_visits[state])

        if np.random.choice(np.arange(self.available_actions), p=[1 - epsilon, epsilon]):
            action = np.random.choice(self.available_actions)  # Explore!
        elif self.state_visits[state] == 0:
            action = 1  # Bias toward going forward
        else:
            action = np.argmax([self.qw(state, act) for act in range(self.available_actions)])

        self.state_visits[state] += 1


        return action

    def update_Q(self, old_s, new_s, old_a, new_a, reward, E):
        delta = reward + self.gamma * self.qw(new_s, new_a) - self.qw(old_s, old_a)
        self.weights += self.alpha * delta * self.get_features(new_s, action)
        # alpha = (1 / self.Nsa[old_s][old_a])

        # for s, A in E.items():
        #     for a in range(len(A)):
        #         self.Q[old_s][old_a] = self.Q[old_s][old_a] + alpha * delta * E[s][a]
        #         E[s][a] = self.gamma * self.alpha * E[s][a]

In [29]:
env, initial_state = environment.get_env()


In [67]:
GAMMA = 0.99

AVAILABLE_STATES = len(RAM_mask)
AVAILABLE_ACTIONS = 2
N0 = 2.5
ALPHA = 0.00001

reward_policy.REWARD_IF_CROSS = 50
reward_policy.REWARD_IF_COLISION = -1
reward_policy.REWARD_IF_STILL = -0.1

agent = SarsaLFA(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, discount_factor=1, alpha=ALPHA)

In [68]:

scores = []
total_rewards = []

In [ ]:
%%time
n_runs = 100

for i in range(n_runs):
    render = i % 200 == 199

    E = defaultdict(lambda: np.zeros(2)) # TODO available actions

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = agent.act(state)
    
    score = 0
    total_reward = 0

    while not game_over:
        if render:
            time.sleep(0.005)
            env.render()

        old_state = state
        old_action = action
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()

        action = agent.act(state)  # Next action
        
        E[old_state][old_action] += 1

        agent.update_Q(old_s=old_state, new_s=state, old_a=old_action, new_a=action, reward=reward, E=E)

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

Run [   0] - Total reward: -168.50 Mean scores: 12.64 Means Scores[:-10]: 12.50 Score: 12 
Run [  10] - Total reward: -158.50 Mean scores: 13.12 Means Scores[:-10]: 13.80 Score: 15 
Run [  20] - Total reward: -144.80 Mean scores: 13.62 Means Scores[:-10]: 14.80 Score: 16 
Run [  30] - Total reward: -123.30 Mean scores: 14.32 Means Scores[:-10]: 16.70 Score: 18 
Run [  40] - Total reward: -132.50 Mean scores: 14.93 Means Scores[:-10]: 17.60 Score: 17 
Run [  50] - Total reward: -117.60 Mean scores: 15.36 Means Scores[:-10]: 17.70 Score: 18 
Run [  60] - Total reward: -124.20 Mean scores: 15.69 Means Scores[:-10]: 17.80 Score: 18 
Run [  70] - Total reward: -113.40 Mean scores: 15.92 Means Scores[:-10]: 17.60 Score: 18 


In [49]:
agent.weights

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan])

In [ ]:
!git commit -